Taken from this excellent video series : https://www.youtube.com/watch?v=qaDe0qQZ5AQ&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=8

## load libraries

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as dataset
import torchvision.transforms as transforms

import pandas as pd
import numpy as np


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
in_channels = 3
num_classes = 10
learning_rate = 1e-3
batch_size = 1024
num_epochs = 5

In [4]:
# datahead = pd.read_csv("../input/overheadmnist/version2/train.csv")

# datahead['label'].unique()

In [5]:
# test defined cnn
class SimpleCNN(nn.Module):
    def __init__(self, in_channels=in_channels, num_classes=num_classes):
        super(SimpleCNN, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=in_channels, out_channels=8,kernel_size=(3,3),padding=(1,1), stride=(1,1))
        self.cnn2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), padding=(1,1), stride=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.fc1 = nn.Linear(16*7*7, num_classes)
    
    def forward(self,x):
        out = F.relu(self.cnn1(x))
        out = self.pool(out)
        out = F.relu(self.cnn2(out))
        out = self.pool(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        return out

## Importing external model

In [6]:
# import sys

model = torchvision.models.vgg16(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    
model


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Changing the model

In [7]:
# this doesnt do anything
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    
    def forward(self, x):
        return x

In [8]:
model.avgpool = Identity()
model.classifier = nn.Sequential(nn.Linear(512, 100), nn.ReLU(), nn.Linear(100,num_classes))
# model

# just like that, we modified default model!

In [9]:
model.features[0] = nn.Conv2d(in_channels, 64, kernel_size=(3,3), padding=(1,1), stride=(1,1))

In [10]:
model.to(device=device)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
# model = SimpleCNN().to(device=device)

In [12]:
# testing
x = torch.randn(batch_size, in_channels, 32, 32).to(device=device)
y = model(x)
y.shape

torch.Size([1024, 10])

The VGG model is too big to be used for something like mnist. so we will download cifar

## Defining a dataset

In [13]:

class OverheadMnist(Dataset):
    def __init__(self, datapath):
        super(OverheadMnist, self).__init__()
        
        df = pd.read_csv(datapath)
        self.x = torch.from_numpy(df.iloc[:,1:].values)
        
        self.x = self.x.reshape(self.x.size(0), 1, 28, 28)
        self.x = self.x.float()
        self.y = torch.from_numpy(df.iloc[:,0].values)
        self.y = self.y.long()
        
        self.n_length = len(df)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_length

## Downloading the Cifar dataset

In [14]:
# train_dataset = OverheadMnist('../input/overheadmnist/version2/train.csv')
# test_dataset = OverheadMnist('../input/overheadmnist/version2/test.csv')

train_dataset =dataset.CIFAR10(root='dataset/', train=True, transform=transforms.ToTensor(), download=True )
test_dataset = dataset.CIFAR10(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

0it [00:00, ?it/s]

Extracting dataset/cifar-10-python.tar.gz to dataset/
Files already downloaded and verified


In [15]:
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## training loop

In [16]:
for epoch in range(num_epochs):
    
    for data, target in train_dataloader:
        data = data.to(device=device)
        target = target.to(device=device)
        
        pred = model(data)
        loss = loss_criterion(pred, target)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
    
    print(f"For epoch : {epoch} loss : {loss}")

For epoch : 0 loss : 1.9007295370101929
For epoch : 1 loss : 1.7038981914520264
For epoch : 2 loss : 1.6440407037734985
For epoch : 3 loss : 1.5935883522033691
For epoch : 4 loss : 1.5901787281036377


In [19]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


In [21]:
check_accuracy(train_dataloader, model)
check_accuracy( test_dataloader, model)

Checking accuracy on training data
Got 22243 / 50000 with accuracy 44.49
Checking accuracy on test data
Got 4456 / 10000 with accuracy 44.56
